In [1]:
!pip install pyDOE==0.3.7
!git clone https://github.com/maziarraissi/PINNs.git

In [2]:
import numpy as np
import matplotlib as mpl
#mpl.use('pgf')

def figsize(scale, nplots = 1):
    fig_width_pt = 390.0                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = nplots*fig_width*golden_mean       # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

import matplotlib.pyplot as plt

# I make my own newfig and savefig functions
def newfig(width, nplots = 1):
    fig = plt.figure(figsize=figsize(width, nplots))
    ax = fig.add_subplot(111)
    return fig, ax

def savefig(filename, crop = True):
    if crop == True:
#        plt.savefig('{}.pgf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.pdf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.eps'.format(filename), bbox_inches='tight', pad_inches=0)
    else:
#        plt.savefig('{}.pgf'.format(filename))
        plt.savefig('{}.pdf'.format(filename))
        plt.savefig('{}.eps'.format(filename))

In [3]:
import sys
import torch
from collections import OrderedDict

from pyDOE import lhs
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time

np.random.seed(1234)

In [4]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [6]:
rho = 1.29
mu = 1.1
gx = -9.81
gy = 0

class PhysicsInformedNN():
    def __init__(self, X_bc1, X_bc2, X_bc3, X_f, U_bc1, layers):  
        
        self.v_bc = torch.tensor(U_bc1[:,0:1]).float().to(device)
        self.p_bc = torch.tensor(U_bc1[:,1:2]).float().to(device)
        
        self.X_bc1_x = torch.tensor(X_bc1[:, 0:1], requires_grad=True).float().to(device)
        self.X_bc1_y = torch.tensor(X_bc1[:, 1:2], requires_grad=True).float().to(device)
        self.X_bc1_t = torch.tensor(X_bc1[:, 2:], requires_grad=True).float().to(device)
        
        self.X_bc2_x = torch.tensor(X_bc2[:, 0:1], requires_grad=True).float().to(device)
        self.X_bc2_y = torch.tensor(X_bc2[:, 1:2], requires_grad=True).float().to(device)
        self.X_bc2_t = torch.tensor(X_bc2[:, 2:], requires_grad=True).float().to(device)
        
        self.X_bc3_x = torch.tensor(X_bc3[:, 0:1], requires_grad=True).float().to(device)
        self.X_bc3_y = torch.tensor(X_bc3[:, 1:2], requires_grad=True).float().to(device)
        self.X_bc3_t = torch.tensor(X_bc3[:, 2:], requires_grad=True).float().to(device)
        
        self.X_f_x = torch.tensor(X_f[:, 0:1], requires_grad=True).float().to(device)
        self.X_f_y = torch.tensor(X_f[:, 1:2], requires_grad=True).float().to(device)
        self.X_f_t = torch.tensor(X_f[:, 2:], requires_grad=True).float().to(device) 

        self.layers = layers
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        
        # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=10000, 
            max_eval=10000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )

        self.iter = 0
                
    def net_u(self, x, y, t):  
        u = self.dnn(torch.cat([x, y, t], dim=1))
        return u
    
    def net_f(self, x, y, t):
        Sol = self.net_u(x,y,t)
        u = Sol[:,0:1]
        v = Sol[:,1:2]
        p = Sol[:,2:]
        
        u_t = torch.autograd.grad(u,t,grad_outputs=torch.ones_like(u),retain_graph=True,create_graph=True)[0]
        v_t = torch.autograd.grad(v,t,grad_outputs=torch.ones_like(v),retain_graph=True,create_graph=True)[0]
        
        u_x = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(u),retain_graph=True,create_graph=True)[0]
        v_x = torch.autograd.grad(v,x,grad_outputs=torch.ones_like(v),retain_graph=True,create_graph=True)[0]
        
        u_xx = torch.autograd.grad(u_x,x,grad_outputs=torch.ones_like(u_x),retain_graph=True,create_graph=True)[0]
        v_xx = torch.autograd.grad(v_x,x,grad_outputs=torch.ones_like(v_x),retain_graph=True,create_graph=True)[0]
        
        u_y = torch.autograd.grad(u,y,grad_outputs=torch.ones_like(u),retain_graph=True,create_graph=True)[0]
        v_y = torch.autograd.grad(v,y,grad_outputs=torch.ones_like(v),retain_graph=True,create_graph=True)[0]
        
        u_yy = torch.autograd.grad(u_y,y,grad_outputs=torch.ones_like(u_y),retain_graph=True,create_graph=True)[0]
        v_yy = torch.autograd.grad(v_y,y,grad_outputs=torch.ones_like(v_y),retain_graph=True,create_graph=True)[0]
        
        p_x = torch.autograd.grad(p,x,grad_outputs=torch.ones_like(p),retain_graph=True,create_graph=True)[0]
        p_y = torch.autograd.grad(p,y,grad_outputs=torch.ones_like(p),retain_graph=True,create_graph=True)[0]
        
        f1 = rho*(u_t+ u*u_x+ v*u_y) + p_x -rho*gx - mu*(u_xx + u_yy)
        f2 = rho*(v_t+ u*v_x+ v*v_y) + p_y -rho*gy - mu*(v_xx + v_yy)
        return f1, f2
    
    def loss_func(self):
        self.optimizer.zero_grad()
        
        Sol1 = self.net_u(self.X_bc1_x, self.X_bc1_y, self.X_bc1_t)
        Sol2 = self.net_u(self.X_bc2_x, self.X_bc2_y, self.X_bc2_t)
        Sol3 = self.net_u(self.X_bc3_x, self.X_bc3_y, self.X_bc3_t)
        
        loss_sol1 = torch.mean((Sol1[:,1:2]-self.v_bc)**2)
        loss_sol1+= torch.mean((Sol1[:,2:]-self.p_bc)**2)
        loss_sol2 = torch.mean((Sol2[:,0:1]) ** 2)
        loss_sol3 = torch.mean((Sol3[:,0:1]) ** 2)        
        
        f1, f2 = self.net_f(self.X_f_x, self.X_f_y, self.X_f_t)
        loss_f1 = torch.mean(f1**2)
        loss_f2 = torch.mean(f2**2)
        
        loss = loss_sol1 + loss_sol2 + loss_sol3 + loss_f1 + loss_f2
        
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            print("Iter:{},Loss_sol1:{},Loss_sol2:{},Loss_sol3:{},Loss_f1:{},Loss_f2:{}".format(self.iter, loss_sol1, loss_sol2, loss_sol3, loss_f1, loss_f2))
        return loss        
            
    def train(self):
        self.dnn.train()
        self.optimizer.step(self.loss_func)
            
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        y = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 2:], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x,y,t)
        f1, f2 = self.net_f(x,y,t)
        u = u.detach().cpu().numpy()
        f1 = f1.detach().cpu().numpy()
        f2 = f2.detach().cpu().numpy()
        return u, f1, f2

In [ ]:
# the physics-guided neural network
class PhysicsInformedNN():
    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x_u = torch.tensor(X_u[:, 0:1], requires_grad=True).float().to(device)
        self.t_u = torch.tensor(X_u[:, 1:2], requires_grad=True).float().to(device)
        self.x_f = torch.tensor(X_f[:, 0:1], requires_grad=True).float().to(device)
        self.t_f = torch.tensor(X_f[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        
        self.layers = layers
        self.nu = nu
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        
        # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )

        self.iter = 0
        
    def net_u(self, x, t):  
        u = self.dnn(torch.cat([x, t], dim=1))
        return u
    
    def net_f(self, x, t):
        """ The pytorch autograd version of calculating residual """
    
        u = self.net_u(x, t)
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = u_t + u * u_x - self.nu * u_xx
        return f
    
    def loss_func(self):
        self.optimizer.zero_grad()
        
        u_pred = self.net_u(self.x_u, self.t_u)
        print(torch.mean(u_pred[:,1:])**2)
        return
        f_pred = self.net_f(self.x_f, self.t_f)
        loss_u = torch.mean((self.u - u_pred) ** 2)
        loss_f = torch.mean(f_pred ** 2)
        
        loss = loss_u + loss_f
        
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
            )
        return loss
    
    def train(self):
        self.dnn.train()
                
        # Backward and optimize
        self.optimizer.step(self.loss_func)

            
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [7]:
N_u = 100
N_f = 10000
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 3]

t = np.array([[t/20] for t in range(10)])
x = np.array([[t/5000] for t in range(500)])
y = np.array([[t/1250] for t in range(-100,100)])

BC1 = np.array([[0,t/1250,0] for t in range(-100,100)])
BC2 = np.array([[a/5000,0.0792,b/200] for a in range(500) for b in range(100)])
BC3 = np.array([[a/5000,-0.08,b/200] for a in range(500) for b in range(100)])

UC1 = np.array([[0.8,0.8]]*200)

lb = np.array([0.0, -0.08, 0.0])
ub = np.array([0.1, 0.08, 0.5])
Xf_train = lb + (ub-lb)*lhs(3, N_f)

idx = np.random.choice(BC1.shape[0], N_u, replace=False)
BC1 = BC1[idx, :]
UC1 = UC1[idx,:]

idx = np.random.choice(BC2.shape[0], 1000, replace=False)
BC2 = BC2[idx, :]
BC3 = BC3[idx, :]

model = PhysicsInformedNN(BC1, BC2, BC3, Xf_train, UC1, layers)
model.train()

In [14]:
X = np.array([[x/5000,y/1250,0.1] for x in range(500) for y in range(-100,100)])
U, F1, F2 = model.predict(X)

In [15]:
U.shape